Imports:

In [ ]:
import os
import pandas as pd
import re
import numpy as np

Filenames and paths:

In [ ]:
#path for input and output
inputpath = './exampleData/inputs'
outputpath = './exampleData'


#filepath and name for metabolite file
f_met = inputpath + '/' + 'sampleTransformationFile.txt'

#filepath and name for peak lists
f1name = inputpath +'/' + 'Time0.xlsx'
f2name = inputpath + '/' + 'Time5.xlsx'

Import metabolite list and calculate element columns to match format:

In [ ]:
#import dataframe of metabolites
df_met = pd.read_csv(f_met,sep="\t")
df_met.rename(columns = {'no':'Formula'}, inplace = True) 

#add columns for each element
df_met['C'] = pd.Series([0] * len(df_met.index))
df_met['H'] = pd.Series([0] * len(df_met.index))
df_met['N'] = pd.Series([0] * len(df_met.index))
df_met['O'] = pd.Series([0] * len(df_met.index))
df_met['S'] = pd.Series([0] * len(df_met.index))
df_met['P'] = pd.Series([0] * len(df_met.index))

#iterate through dataframe (should vectorize for better performance) and populate element columns
#turn off warning
pd.options.mode.chained_assignment = None  # default='warn'
for row in range(len(df_met.index)):
    a = re.findall(r'([A-Z][a-z]*)(\d*)', df_met['Formula'][row])
    for b in a:
        if b[1] == '':
            df_met[b[0]][row] = 1
        else:
            df_met[b[0]][row] = int(b[1])
print('Metabolites:')
print(df_met)

Do subtraction and comparison:

In [ ]:
#import two peak lists
df1 =  pd.read_excel(f1name)
df2 =  pd.read_excel(f2name)

#output file
f_out_name = outputpath + '/' + 'CountForEachMetabolite.csv'
f_out = open(f_out_name,'w')
f_out.write('Name,Count\n')

#slice dataframes into only columns to compare
df1comp = df1.loc[:, ['C', 'H', 'N','O','S','P']]
df2comp = df1.loc[:, ['C', 'H', 'N','O','S','P']]
df_metcomp = df_met.loc[:, ['C', 'H', 'N','O','S','P']]

for row in range(len(df_met.index)):
    met = df_metcomp.loc[row]
    df1sub = df1comp-met
    df = pd.merge(df1sub, df2comp, how='left', indicator='Exist')
    df['Exist'] = np.where(df.Exist == 'both', True, False)
    matches = df1[df.Exist==True]
    print(matches)
    name = outputpath + '/met' + str(row) + '_' + df_met['Isomeric'][row] + '.csv'
    matches.to_csv(name)
    print(df_met.loc[row][0],'\t',len(matches.index))
    f_out.write(str(df_met.loc[row][0])+','+str(len(matches.index))+'\n')
   
f_out.close()